In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from pysilsub.problem import SilentSubstitutionProblem as SSP
from pysilsub.CIE import get_CIES026

In [2]:
# Functions for stimulus waveform
def get_time_vector(duration):
    t = np.arange(0, (duration*1000), 10).astype("int")
    return t


def sinusoid_modulation(f, duration, Fs=50):
    x = np.arange(duration * Fs)
    sm = np.sin(2 * np.pi * f * x / Fs)
    return sm


def modulate_intensity_amplitude(sm, background, amplitude):
    ivals = (background + (sm*amplitude)).astype("int")
    return ivals

In [3]:
# Load the calibration data
spds = pd.read_csv(
    '../data/BCGAR_5_Primary_8_bit_linear.csv', 
    index_col=['Primary','Setting'])
spds.columns = pd.Int64Index(spds.columns.astype(int))
spds.columns.name = 'Wavelength'

# List of colors for the primaries
colors = ['blue', 'cyan', 'green', 'orange', 'red'] 


ssp = SSP(
    resolutions=[255]*5,  # Five 8-bit primaries 
    colors=colors,  # Colors of the LEDs
    spds=spds,  # The calibration data
    wavelengths=[380, 781, 1],  # SPD wavelength binwidth
    ignore=['R'],  # Ignore rods
    silence=['M', 'L', 'I'],  # Silence S-, M-, and L-cones
    isolate=['S'],  # Isolate melanopsin
    target_contrast=2.,  # Aim for 250% contrast 
    name='BCGAR (8-bit, linear)',  # Description of device
    background=[.5]*5
) 

#spd_fig = ssp.plot_spds()

TypeError: __init__() got an unexpected keyword argument 'spds'

In [ ]:
# Target contrast vals for modulation
contrast_waveform = sinusoid_modulation(f=1, duration=1, Fs=50) * 1.
plt.plot(contrast_waveform)
peak = np.argmax(contrast_waveform)
trough = np.argmin(contrast_waveform)
target_contrasts = contrast_waveform[peak:trough+1]
plt.plot(np.hstack([target_contrasts, target_contrasts[::-1]]))

S-cone contrast modulation
==========================

In [ ]:
# Calcualte modulation spectra for S-cone modulation
contrast_mods = [ssp.linalg_solve([tc, 0, 0, 0]) for tc in target_contrasts]

plt.plot(ssp.predict_multiprimary_spd(contrast_mods[0]), lw=1, label='+ve')
plt.plot(ssp.predict_multiprimary_spd(ssp.background), lw=1, label='background')
plt.plot(ssp.predict_multiprimary_spd(contrast_mods[-1]), lw=1, label='-ve')
plt.legend()


Modulation peak
---------------

In [ ]:
ssp.plot_ss_result(contrast_mods[0])
contrasts = ssp.get_photoreceptor_contrasts(contrast_mods[0])
print(f'S-cone contrast: {contrasts[0]}')
print(f'Splatter: {contrasts[1:]}')

Modulation trough
-----------------

In [ ]:
ssp.plot_ss_result(contrast_mods[-1])
contrasts = ssp.get_photoreceptor_contrasts(contrast_mods[-1])
print(f'S-cone contrast: {contrasts[0]}')
print(f'Splatter: {contrasts[1:]}')

Contrast modulations
--------------------

In [ ]:
palette = sns.diverging_palette(220, 20, n=len(contrast_mods), l=65, as_cmap=False)
bg_spd = ssp.predict_multiprimary_spd(ssp.background)
for i, s in enumerate(contrast_mods):
    mod_spd = ssp.predict_multiprimary_spd(s) 
    plt.plot(mod_spd-bg_spd, c=palette[i], lw=1)
    
plt.xlabel('Wavelength (nm)')
plt.ylabel('S-cone contrast (%)');

In [ ]:
splatter = [ssp.get_photoreceptor_contrasts(cm) for cm in contrast_mods]
splatter = np.vstack(splatter)

plt.plot(splatter[:,0], label='S', c='b')
plt.plot(splatter[:,1], label='M', c='g')
plt.plot(splatter[:,2], label='L', c='r')
plt.plot(splatter[:,3], label='R', c='k')
plt.plot(splatter[:,4], label='I', c='cyan')
plt.ylabel('Simple contrast')
plt.legend()

In [ ]:
background = ssp.predict_multiprimary_spd(ssp.background)
background.index

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import animation
%matplotlib widget

background = ssp.predict_multiprimary_spd(ssp.background)
contrast_mods = contrast_mods + contrast_mods[::-1]
# First set up the figure, the axis, and the plot element we want to animate
fig = plt.figure()
ax = plt.axes(xlim=(300, 800), ylim=(-.05, .8))
bg_spd, = ax.plot([], [], lw=2, color='k', label='Background SPD')
spd, = ax.plot([], [], lw=2, color='k', ls=':', label='SPD')
p1, = ax.plot([], [], lw=2, color='blue', label='Primary 1')
p2, = ax.plot([], [], lw=2, color='cyan', label='Primary 2')
p3, = ax.plot([], [], lw=2, color='green', label='Primary 3')
p4, = ax.plot([], [], lw=2, color='orange', label='Primary 4')
p5, = ax.plot([], [], lw=2, color='red', label='Primary 5')
ax.legend()

# initialization function: plot the background of each frame

def init():
    bg_spd.set_data([], [])
    spd.set_data([], [])
    p1.set_data([], [])
    p2.set_data([], [])
    p3.set_data([], [])
    p4.set_data([], [])
    p5.set_data([], []) 
    return bg_spd, spd, p1, p2, p3, p4, p5,

# animation function.  This is called sequentially
def animate(i):
    primaries = ssp.predict_multiprimary_spd(contrast_mods[i], nosum=True)
    spec = ssp.predict_multiprimary_spd(contrast_mods[i])
    
    a = primaries[0]
    b = primaries[1]
    c = primaries[2]
    d = primaries[3]
    e = primaries[4]
    
    bg_spd.set_data(background.index, background.values)
    spd.set_data(background.index, spec.values)
    p1.set_data(background.index, a.values)
    p2.set_data(background.index, b.values)
    p3.set_data(background.index, c.values)
    p4.set_data(background.index, d.values)
    p5.set_data(background.index, e.values)

    return bg_spd, spd, p1, p2, p3, p4, p5,

# call the animator.  blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=len(contrast_mods), 
                               interval=20, blit=True)


# save the animation as an mp4.  This requires ffmpeg or mencoder to be
# installed.  The extra_args ensure that the x264 codec is used, so that
# the video can be embedded in html5.  You may need to adjust this for
# your system: for more information, see
# http://matplotlib.sourceforge.net/api/animation_api.html
anim.save('basic_animation.mp4', fps=30, extra_args=['-vcodec', 'libx264'])
anim.save('basic_animation.gif', fps=30, writer='imagemagick')

plt.show()